In [1]:
from pprint import pprint
from pathlib import Path
import numpy as np
import imageio

from IPython.display import Markdown as md

from turbx import REPO_PATH, log
from turbx.data import DataLoader, Dataset
from turbx.filter import dft, common

log.setLevel('WARN')

[DEBUG] 08/12/2022 04:32:04PM: MainProcess: __init__.py - Loaded logging config file: /home/nowa201/Projects/fish_detection/fish/src/turbx/logging.yaml


**Approach**
1. Create dataloader and fetch video,label pairs
2. Create list of (boxes, track, etc.) for each frame
3. Draw boxes on each frame
4. Write video to file as gif (standard name/location)
5. Display raw, filter, etc side by side in markdown
6. Calculate and plot metric

In [2]:
# create test output directory
file_path = Path('').absolute()
out_path = Path(f"{file_path}/outputs/{str(file_path.name).split('.')[0]}")
out_path.mkdir(parents=True, exist_ok=True)

file_path = f"{REPO_PATH}/data/mp4"
labels = f"{REPO_PATH}/data/labels"

In [3]:
dataloader = DataLoader(Dataset(videos=file_path, labels=labels))

In [4]:
# create filter class used in each test
fps = 10
dft_filter = dft.DFTFilter(fps=fps)
mean_filter = common.MeanFilter(fps=fps)
intensity_filter = common.IntensityFilter(fps=fps)

In [5]:
# get next; calculate filter
video, label = next(dataloader)

# generate each filter
mean_filter.calculate(video, fps)
print("Mean Filter done")
dft_filter.calculate(video, fps)
print("DFT Filter done")
intensity_filter.calculate(video, fps)
print("Intensity Filter done")

Mean Filter done
DFT Filter done
Intensity Filter done


In [6]:
# apply each filter
mean_out = mean_filter.filter(video)
dft_mean_out = dft_filter.filter(mean_out)
int_dft_mean_out = intensity_filter.filter(dft_mean_out)

assert isinstance(int_dft_mean_out, np.ndarray), "filter output is not numpy array"

In [7]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML
fig = plt.figure()
im = plt.imshow(video[0,...])
plt.close()

def init_anim():
    im.set_data(video[0,...])
def animate(i):
    im.set_data(video[i, ...])
    return im

anim = animation.FuncAnimation(fig, animate, init_func=init_anim, frames=video.shape[0],
                               interval=10)
HTML(anim.to_html5_video()))

TypeError: Markdown expects text, not <IPython.core.display.HTML object>

In [ ]:
# save filtered video

In [ ]:
# calculate video score

In [ ]:
# create animated video graph

In [ ]:
# compose multipane gif

Display gifs using Markdown

<img src="../experiments/dft/outputs/all_ac_video/demo_raw_video.gif" alt="raw_video" height="500"/>
